# Clustering Crypto

In [3]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
# !pip install hvplot
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [4]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
coinlist_data = requests.get(url).json()

In [5]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
coinlist_df = pd.DataFrame(coinlist_data["Data"]).T
coinlist_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,AssetLaunchDate,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,2017-07-01,300,0,0,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")
file_path = Path("Resources/crypto_data.csv")
# Creating a DataFrame
crypto_df = pd.read_csv(file_path)
crypto_df
# Despite loading JSON data will go for the CSV file

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


### Data Preprocessing

In [7]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df.set_index("Unnamed: 0", inplace = True)
crypto_df.index.name = None
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [8]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df["IsTrading"] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [9]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df['Algorithm'].isnull() == False]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [10]:
# Remove the 'IsTrading' column
crypto_df.drop(columns=['IsTrading'], inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [11]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)

In [12]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] > 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [13]:
# Drop rows where there are 'N/A' text values
crypto_df.drop(crypto_df.index[(crypto_df['CoinName'] == 'N/A') |
           (crypto_df['Algorithm'] == 'N/A') |
           (crypto_df['ProofType'] == 'N/A') |
           (crypto_df['TotalCoinsMined'] == 'N/A') | 
           (crypto_df['TotalCoinSupply'] == 'N/A')], inplace = True)
crypto_df.size

2660

In [14]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coin_names_df = crypto_df[["CoinName"]]
coin_names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [15]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop("CoinName", axis=1, inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [16]:
# Create dummy variables for text features
LE = LabelEncoder()
X = crypto_df.copy()
X['Algorithm'] = LE.fit_transform(X['Algorithm']) 
X['ProofType'] = LE.fit_transform(X['ProofType']) 
X.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,52,15,4.199995e+01,42
404,52,15,1.055185e+09,532000000
1337,66,15,2.927942e+10,314159265359
BTC,47,12,1.792718e+07,21000000
ETH,20,12,1.076842e+08,0


In [17]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[ 3.76459118e-01  8.91356555e-01 -1.17108170e-01 -1.52870298e-01]
 [ 3.76459118e-01  8.91356555e-01 -9.39695522e-02 -1.45008997e-01]
 [ 1.21543803e+00  8.91356555e-01  5.24945609e-01  4.48942416e+00]
 [ 7.68237937e-02  1.67233875e-03 -1.16715055e-01 -1.52559984e-01]
 [-1.54120696e+00  1.67233875e-03 -1.14746818e-01 -1.52870298e-01]]


### Reducing Dimensions Using PCA

In [18]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [19]:
# Create a DataFrame with the principal components data
X_pca = pca.fit_transform(X_scaled)
pcs_df = pd.DataFrame(data= X_pca, index=X.index, columns = ["PC 1", "PC 2","PC 3"])
pcs_df.head(10) 

# Examining the explained variance
pca.explained_variance_ratio_

array([0.4350467 , 0.28670411, 0.20538956])

### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [20]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=6`

In [22]:
# Initialize the K-Means model
k=6
model = KMeans(n_clusters=k, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["Class"] = model.labels_

pcs_df.head(10)

,PC 1,PC 2,PC 3,Class
42,-0.417875,0.810296,0.372138,0
404,-0.396564,0.815135,0.373256,0
1337,3.124076,2.209780,0.504335,5
BTC,-0.192083,0.016266,-0.072910,1
ETH,-0.044116,-1.167492,1.012525,4
LTC,-0.217954,0.235824,-0.273789,1
DASH,-0.482771,1.336609,-0.110253,0
XMR,-0.001979,-1.518631,1.334156,4
ETC,-0.041907,-1.167016,1.012991,4
ZEC,-0.034542,-1.255502,1.092984,4


In [27]:
# Creating a new DataFrame named clustered_df, that includes the following columns "Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "PC 1", "PC 2", "PC 3", "CoinName", "Class"
clustered_df = crypto_df.merge(pcs_df, how='outer', left_index=True, right_index=True)
clustered_df['CoinName'] = coin_names_df.values
reorder = ['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply', 'PC 1', 'PC 2', 'PC 3', 'CoinName', 'Class']
clustered_df = clustered_df.reindex(columns=reorder)
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.417875,0.810296,0.372138,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.396564,0.815135,0.373256,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,3.124076,2.209780,0.504335,EliteCoin,5
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.192083,0.016266,-0.072910,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.044116,-1.167492,1.012525,Ethereum,4
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.217954,0.235824,-0.273789,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.482771,1.336609,-0.110253,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.001979,-1.518631,1.334156,Monero,4
ETC,Ethash,PoW,1.133597e+08,210000000,-0.041907,-1.167016,1.012991,Ethereum Classic,4
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.034542,-1.255502,1.092984,ZCash,4


### Visualizing Results

#### 3D-Scatter with Clusters

In [43]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    color="Class",
    symbol="Class",
    width=800)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [31]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', "TotalCoinsMined", 'Class'], width=1200)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [35]:
# Print the total number of tradable cryptocurrencies
print(f"The total number of tradable cryptocurrencies is {len(clustered_df.index)}.")

The total number of tradable cryptocurrencies is 532.


#### Scatter Plot with Tradable Cryptocurrencies

In [41]:
# Scale data to create the scatter plot
column_names_to_normalize = ['TotalCoinsMined', 'TotalCoinSupply'] 
x_data = clustered_df[column_names_to_normalize].values
x_data_scaled = min_max_scaler.fit_transform(x_data)
clustered_df_scaled = pd.DataFrame(x_data_scaled, columns=column_names_to_normalize, index = clustered_df.index)
clustered_df[column_names_to_normalize] = clustered_df_scaled
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,0.000000,4.200000e-11,-0.417875,0.810296,0.372138,42 Coin,0
404,Scrypt,PoW/PoS,0.001066,5.320000e-04,-0.396564,0.815135,0.373256,404Coin,0
1337,X13,PoW/PoS,0.029576,3.141593e-01,3.124076,2.209780,0.504335,EliteCoin,5
BTC,SHA-256,PoW,0.000018,2.100000e-05,-0.192083,0.016266,-0.072910,Bitcoin,1
ETH,Ethash,PoW,0.000109,0.000000e+00,-0.044116,-1.167492,1.012525,Ethereum,4
LTC,Scrypt,PoW,0.000064,8.400000e-05,-0.217954,0.235824,-0.273789,Litecoin,1
DASH,X11,PoW/PoS,0.000009,2.200000e-05,-0.482771,1.336609,-0.110253,Dash,0
XMR,CryptoNight-V7,PoW,0.000017,0.000000e+00,-0.001979,-1.518631,1.334156,Monero,4
ETC,Ethash,PoW,0.000115,2.100000e-04,-0.041907,-1.167016,1.012991,Ethereum Classic,4
ZEC,Equihash,PoW,0.000007,2.100000e-05,-0.034542,-1.255502,1.092984,ZCash,4


In [42]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot.scatter(
x="TotalCoinsMined",
y="TotalCoinSupply", 
xlim = (-0.05, 1.1),
ylim = (-0.05, 1.1),
hover_cols=["CoinName"],
by="Class")

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)